In [ ]:
import toml
import re
import urllib
import pygsheets
import benchlingapi
import requests_html

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import paulssonlab.api as api
import paulssonlab.cloning.workflow as workflow

# Code

# Setup

In [ ]:
config = toml.load("config.toml")

In [ ]:
session = benchlingapi.Session(config["benchling"]["api_key"])

In [ ]:
gc = pygsheets.authorize(service_account_file="credentials.json")

In [ ]:
col = workflow.get_strain_collection_sheets(gc.drive.service, "LIB")
col

# Google Sheets

In [ ]:
PAULSSON_GDRIVE = "157G6iXC8pfXnI4HoiiRItWc5awDPNhmb"

In [ ]:
api.google.get_drive_by_path(
    gc.drive.service, "Strains/LIB_Collection", root=PAULSSON_GDRIVE
)

In [ ]:
workflow.get_next_collection_id(gc.open_by_key(col["strains"]).worksheet())

In [ ]:
workflow.get_next_collection_id(gc.open_by_key(col["oligos"]).worksheet())

In [ ]:
workflow.get_next_collection_id(gc.open_by_key(col["plasmids"]).worksheet())

In [ ]:
workflow.get_next_collection_id(
    gc.open_by_key(col["parts"]).worksheet_by_title("Parts")
)

In [ ]:
workflow.get_next_collection_id(gc.open_by_key(col["parts"]).worksheet_by_title("TUs"))

In [ ]:
workflow.get_next_collection_id(
    gc.open_by_key(col["parts"]).worksheet_by_title("Assemblies")
)

In [ ]:
def get_plasmid_maps():
    pass

In [ ]:
res = (
    gc.drive.service.files()
    .list(
        q=f"('{col['plasmid_maps']}' in parents) and (mimeType != 'application/vnd.google-apps.folder')"
    )
    .execute()
)

# Benchling/Addgene

In [ ]:
folder = api.get_folder(session, "Test2", project="Test")

In [ ]:
dna = session.DNASequence.find_by_name("V37m", folder_id=folder.id)

In [ ]:
# url = "https://www.addgene.org/108254/"
url = "https://www.addgene.org/108537/"
# url = 'http://www.addgene.org/121010/'
api.addgene.get_addgene(url)

In [ ]:
# url = 'https://www.addgene.org/kits/marionette-sensor-collection/#kit-contents'
url = "http://www.addgene.org/kits/murray-cidar-moclo-v1/"
kit = api.addgene.get_addgene(url, include_details=True, include_sequences=True)

In [ ]:
kit["wells"][0]

In [ ]:
gb = api.get_genbank(
    "https://media.addgene.org/snapgene-media/v1.6.2-0-g4b4ed87/sequences/05/72/230572/addgene-plasmid-120934-sequence-230572.gbk"
)

In [ ]:
# gb = get_genbank(addgene_sequences('http://www.addgene.org/121010/')['depositor_full'][0])[0]
gb = get_genbank(
    addgene_sequences("https://www.addgene.org/108537/")["addgene_full"][0]
)[0]

# Genbank parsing

# Benchling

In [ ]:
dna = genbank_to_benchling(session, gb, "seq3", folder.id, accession="addgene_blah")
dna.save()

# Densmore/3G/JUMP

In [ ]:
densmore_kit = "http://www.addgene.org/kits/densmore-cidar-moclo/"
threeg_kit = "http://www.addgene.org/kits/murray-cidar-moclo-v1/"
marionette_kit = "https://www.addgene.org/kits/marionette-sensor-collection/"
jump_plasmids = [
    "https://www.addgene.org/126956/",
    "https://www.addgene.org/126959/",
    "https://www.addgene.org/126960/",
    "https://www.addgene.org/126961/",
    "https://www.addgene.org/126962/",
    "https://www.addgene.org/126963/",
    "https://www.addgene.org/126964/",
    "https://www.addgene.org/126965/",
    "https://www.addgene.org/126966/",
    "https://www.addgene.org/126967/",
    "https://www.addgene.org/126973/",
    "https://www.addgene.org/126974/",
    "https://www.addgene.org/126975/",
    "https://www.addgene.org/126976/",
    "https://www.addgene.org/126991/",
    "https://www.addgene.org/126996/",
    "https://www.addgene.org/127015/",
    "https://www.addgene.org/127047/",
    "https://www.addgene.org/127051/",
    "https://www.addgene.org/127025/",
    "https://www.addgene.org/127000/",
]

In [ ]:
strain_sheet = gc.open_by_key(col["strains"]).worksheet()
plasmid_sheet = gc.open_by_key(col["plasmids"]).worksheet()

In [ ]:
x = workflow.import_addgene(
    densmore_kit, strain_sheet, plasmid_sheet, col["plasmid_maps"]
)

In [ ]:
y = workflow.import_addgene(
    threeg_kit, strain_sheet, plasmid_sheet, col["plasmid_maps"]
)

In [ ]:
z = workflow.import_addgene(
    marionette_kit, strain_sheet, plasmid_sheet, col["plasmid_maps"]
)

In [ ]:
zz = workflow.import_addgene(
    jump_plasmids, strain_sheet, plasmid_sheet, col["plasmid_maps"]
)

In [ ]:
addgene_plasmids = [
    f"https://www.addgene.org/{catalog}/" for catalog in addgene_catalog
] + [densmore_kit]

In [ ]:
addgene_data = workflow.import_addgene(
    addgene_plasmids, strain_sheet, plasmid_sheet, col["plasmid_maps"]
)

# Lab Obs ordering site

## Test

In [ ]:
import paulssonlab.api.labops as labops

In [ ]:
browser = await labops.login_harvard(
    config["ecommons"]["username"],
    config["ecommons"]["password"],
    service="https://sysbiolabops.hms.harvard.edu/casservice",
)

In [ ]:
addgene_orders = await labops.get_orders(browser, vendor="Addgene (26)")

In [ ]:
my_addgene = addgene_orders[
    addgene_orders["Name"].isin(["Jacob Shenker", "Noah Olsman"])
]["Catalog #"]

In [ ]:
addgene_catalog = my_addgene.apply(
    lambda x: int(re.search(r"(\d+)$", x).group(1))
).values

In [ ]:
addgene_orders[~addgene_orders["Name"].isin(["Jacob Shenker", "Daniel Eaton"])]

In [ ]:
addgene_orders["Name"].value_counts()

## All orders

In [ ]:
prices = df.loc[:, "Total Price"]
prices = prices.str.replace(",", "")
prices = prices.str.replace("$", "")
prices = prices.astype(float)

In [ ]:
prices.hist(bins=100, log=True)